In [339]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import seaborn as sns
from matplotlib import style
import math
import random

In [340]:
#LP MAXIMIZATION MODEL

from pulp import *
Promo_Model = LpProblem("Pharma_Promotion_Model", LpMaximize)

In [341]:
#PARAMETERS

T=12 # end of promotion horizon
PHorizon = list(range(0, T))

I=10 # product count 
Products = list(range(0, I))

#NOT USED RIGHT NOW
Promo_Levels = [0,1,2]

Demand = np.zeros((I, T), dtype = int)

for i in Products:
    for t in PHorizon:
       Demand[i][t] = random.randint(0, 10000)
    
Total_PaidQty_Limit = 1000000
Total_FreeQty_Limit = 250000

Demand

array([[4837, 4626, 3045, 4842, 8876, 2626, 8880, 4146, 3347, 6300, 4654,
        1079],
       [4789, 8436, 3105, 4107, 6743, 3418, 6131, 1084, 9592, 7582, 3606,
        6797],
       [3175, 4944, 3869, 1891, 4163, 9500, 2030, 3906, 4375, 5604, 5778,
        5689],
       [9760, 4457, 7789, 9787, 3790, 9262, 6999, 1091, 8840, 6854, 3785,
        9200],
       [1995, 7827, 4249, 2984, 2703, 7172, 5290, 7784, 7623, 9143, 2314,
        5473],
       [ 327, 8427, 3250,  706, 5878, 1038, 8703, 3811, 9465, 1066, 4530,
         751],
       [8319,  220, 9102, 1202, 8212, 7931, 7658, 3833, 6211, 4768,  665,
        9031],
       [ 770, 3693, 3454,   26, 1046, 3027, 8992, 1193, 8524,  408, 6136,
         410],
       [5547, 7449, 7872, 2565, 8315, 1203, 4737, 5918, 2294, 7932, 7055,
        8698],
       [7970, 7510, 3846, 9423, 3756, 7761, 4134, 1395,  933, 1957, 8485,
        1021]])

In [342]:
#DECISION VARIABLES

#Paid Quantity
PaidQtyLimit = 10000
P = LpVariable.dicts("Paid_Quantity",(Products,PHorizon),lowBound=0, upBound=PaidQtyLimit, cat='Integer')

#Free Quantity
FreeQtyLimit = 5000
F = LpVariable.dicts("Free_Quantity",(Products,PHorizon),lowBound=0, upBound=FreeQtyLimit, cat='Integer')

#Promo Ratio
#R = LpVariable.dicts("Promo_Ratio",(Products,PHorizon),lowBound=0, upBound=1, cat='Continuous')

#Is Product "i" at Time "t" Promoted?
X = LpVariable.dicts("IsPromoted",(Products,PHorizon),cat='Binary')

In [343]:
#OBJECTIVE FUNCTION

Total_Paid_Quantity = lpSum(lpSum(P[i][t] for i in Products) for t in PHorizon)

Promo_Model += Total_Paid_Quantity

In [344]:
#DEMAND SATISFACTION
for i in Products:
    for t in PHorizon:
       Promo_Model += P[i][t] + F[i][t] == Demand[i][t]*(1 + F[i][t]*(1/FreeQtyLimit))
    
#NO FREE WITHOUT PROMOTION
M = FreeQtyLimit
for i in Products:
    for t in PHorizon:
       Promo_Model += F[i][t] <= X[i][t] * M
    
#PROMO RATIO
#for i in Products:
#    for t in PHorizon:
#        Promo_Model += F[i][t] == R[i][t]*Demand[i][t]      

#PROMOTED PRODUCTS AT TIME "t"
for t in PHorizon:
    Promo_Model += lpSum(X[i][t] for i in Products) == 5
    
#PROMO PERIOD FOR PRODUCT "i"  
for i in Products:
    Promo_Model += lpSum(X[i][t] for t in PHorizon) == 6

#TOTAL PAID QUANTITY CAPACITY
for i in Products:
    for t in PHorizon:
        Promo_Model += lpSum(P[i][t] for t in PHorizon) <= Total_PaidQty_Limit
    
#TOTAL FREE QUANTITY CAPACITY
for i in Products:
    for t in PHorizon:
        Promo_Model += lpSum(F[i][t] for t in PHorizon) <= Total_FreeQty_Limit

In [345]:
Promo_Model.solve()
# The status of the solution is printed to the screen
print("Status:", LpStatus[Promo_Model.status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/baris/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/11/n43tq6bj255drztgc6ydfmdc0000gn/T/d585016325d543d78c99346ca96efacb-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/11/n43tq6bj255drztgc6ydfmdc0000gn/T/d585016325d543d78c99346ca96efacb-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 507 COLUMNS
At line 4948 RHS
At line 5451 BOUNDS
At line 5812 ENDATA
Problem MODEL has 502 rows, 360 columns and 3600 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 679947 - 0.00 seconds
Cgl0003I 0 fixed, 173 tightened bounds, 63 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 151 tightened bounds, 61 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 152 tightened bounds, 60 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 1

In [346]:
# OUTPUT

# OPTIMIZED OBJECTIVE FUNCTION
print("Optimal Total Paid Quantity = ", pulp.value(Promo_Model.objective),"\n")

# PRINTS VARIABLES AND OPTIMAL VALUES
for v in Promo_Model.variables():
    print(v.name, "=", v.varValue)

Optimal Total Paid Quantity =  651282.0 

Free_Quantity_0_0 = 0.0
Free_Quantity_0_1 = 0.0
Free_Quantity_0_10 = 0.0
Free_Quantity_0_11 = 0.0
Free_Quantity_0_2 = 0.0
Free_Quantity_0_3 = 0.0
Free_Quantity_0_4 = 1250.0
Free_Quantity_0_5 = 0.0
Free_Quantity_0_6 = 1375.0
Free_Quantity_0_7 = 0.0
Free_Quantity_0_8 = 0.0
Free_Quantity_0_9 = 5000.0
Free_Quantity_1_0 = 0.0
Free_Quantity_1_1 = 1250.0
Free_Quantity_1_10 = 0.0
Free_Quantity_1_11 = 5000.0
Free_Quantity_1_2 = 0.0
Free_Quantity_1_3 = 0.0
Free_Quantity_1_4 = 5000.0
Free_Quantity_1_5 = 0.0
Free_Quantity_1_6 = 5000.0
Free_Quantity_1_7 = 0.0
Free_Quantity_1_8 = 0.0
Free_Quantity_1_9 = 2500.0
Free_Quantity_2_0 = 0.0
Free_Quantity_2_1 = 0.0
Free_Quantity_2_10 = 5000.0
Free_Quantity_2_11 = 5000.0
Free_Quantity_2_2 = 0.0
Free_Quantity_2_3 = 0.0
Free_Quantity_2_4 = 0.0
Free_Quantity_2_5 = 550.0
Free_Quantity_2_6 = 0.0
Free_Quantity_2_7 = 0.0
Free_Quantity_2_8 = 0.0
Free_Quantity_2_9 = 5000.0
Free_Quantity_3_0 = 250.0
Free_Quantity_3_1 = 0.0
Fre

In [372]:
#PROMO RATIO IS CALCULATED BASED ON THE OPTIMUM P[i][t] AND F[i][t] VALUES
Promo_Ratio = np.zeros((I, T), dtype = float)

for i in Products:
    for t in PHorizon:
        Promo_Ratio[i][t] = (F[i][t].varValue)/(P[i][t].varValue + F[i][t].varValue)

df_Promo_Ratio = pd.DataFrame(Promo_Ratio,
                 index=range(1,I+1),
                 columns=range(1,T+1))
df_Promo_Ratio

,1,2,3,4,5,6,7,8,9,10,11,12
1,0.000000,0.000000,0.000000,0.0,0.112663,0.000000,0.121445,0.00000,0.000000,0.396825,0.000000,0.000000
2,0.000000,0.118540,0.000000,0.0,0.370755,0.000000,0.407764,0.00000,0.000000,0.219819,0.000000,0.367809
3,0.000000,0.000000,0.000000,0.0,0.000000,0.052157,0.000000,0.00000,0.000000,0.446110,0.432676,0.439445
4,0.024395,0.000000,0.000000,0.0,0.000000,0.000000,0.357194,0.00000,0.130526,0.364751,0.000000,0.086774
5,0.000000,0.000000,0.000000,0.0,0.000000,0.348578,0.000000,0.27529,0.000000,0.000000,0.000000,0.456788
6,0.000000,0.000000,0.000000,0.0,0.425315,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.0,0.202955,0.000000,0.217637,0.00000,0.402512,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.111210,0.00000,0.117316,0.000000,0.407432,0.000000
9,0.000000,0.335616,0.244293,0.0,0.171807,0.000000,0.000000,0.42244,0.000000,0.210119,0.354359,0.000000
10,0.235257,0.315369,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.168364,0.000000
